In [1]:
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
from datasets import load_dataset
dataset = load_dataset("mteb/tweet_sentiment_extraction")
df = pd.DataFrame(dataset['train'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/3.63M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/465k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/27481 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3534 [00:00<?, ? examples/s]

In [3]:
df

,id,text,label,label_text
0,cb774db0d1,"I`d have responded, if I were going",1,neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,0,negative
2,088c60f138,my boss is bullying me...,0,negative
3,9642c003ef,what interview! leave me alone,0,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",0,negative
...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,0,negative
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,0,negative
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,2,positive
27479,ed167662a5,But it was worth it ****.,2,positive


In [4]:

from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
  return(tokenizer(examples["text"],padding="max_length",truncation = True))

tokenized_dataset = dataset.map(tokenize_function, batched = True)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/27481 [00:00<?, ? examples/s]

Map:   0%|          | 0/3534 [00:00<?, ? examples/s]

In [5]:
small_trained_dataset =  tokenized_dataset['train'].shuffle(seed=42).select(range(1000))
small_eval_dataset =  tokenized_dataset['test'].shuffle(seed=42).select(range(1000))

In [6]:
from transformers import GPT2ForSequenceClassification
model = GPT2ForSequenceClassification.from_pretrained("gpt2",num_labels = 3)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
import numpy as np

In [18]:
import evaluate
metric = evaluate.load('accuracy')
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis = 1)
  return metric.compute(predictions = predictions,  references = labels)


In [19]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir = "test_trainer",
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    gradient_accumulation_steps = 4
    )
trainer = Trainer(model = model,
                  args = training_args,
                  train_dataset = small_trained_dataset,
                  eval_dataset = small_eval_dataset,
                  compute_metrics= compute_metrics,
                  )

trainer.train()

Step,Training Loss
500,0.590400


TrainOutput(global_step=750, training_loss=0.5007161763509115, metrics={'train_runtime': 841.2956, 'train_samples_per_second': 3.566, 'train_steps_per_second': 0.891, 'total_flos': 1567794659328000.0, 'train_loss': 0.5007161763509115, 'epoch': 3.0})

This output summarizes the results of training a model using the Trainer class from the transformers library. Here’s a brief explanation of each part:

1. **global_step=750**: The model completed 750 training steps (iterations).
2. **training_loss=1.1669:** The average loss during training is 1.1669. Lower loss values generally indicate better model performance, though the value depends on the task.
3. **metrics:**
  a. *train_runtime=846.19 seconds*: The total time taken for training is around 846 seconds (approximately 14 minutes).
  b. *train_samples_per_second=3.545*: The model processed 3.545 samples per second during training.
  c. *train_steps_per_second=0.886:* The model processed 0.886 steps per second (each step typically includes a batch of samples).
  d. *total_flos=1.5678e15:* This represents the total floating-point operations (FLOPs) performed during training (a measure of computational work).
  e. *train_loss=1.1669:* This confirms the average training loss over the training process.
  f. *epoch=3.0:* The training completed after 3 full epochs (passes through the entire dataset).
  
  **Summary**:
The model completed 750 training steps across 3 epochs, with an average training loss of 1.1669. The training took around 846 seconds, processing approximately 3.5 samples per second.


In [20]:
trainer.evaluate()

{'eval_loss': 1.781160831451416,
 'eval_accuracy': 0.691,
 'eval_runtime': 69.6435,
 'eval_samples_per_second': 14.359,
 'eval_steps_per_second': 14.359,
 'epoch': 3.0}

### Evaluation Results

- **Evaluation Loss**: `1.7812`
  - This measures the model's performance on the test set. A lower loss indicates better performance.
  
- **Evaluation Accuracy**: `0.691`
  - The model achieved an accuracy of 69.1%, meaning that 69.1% of predictions were correct.

- **Evaluation Runtime**: `69.64 seconds`
  - The total time taken for evaluation was approximately 69.64 seconds.

- **Samples Processed Per Second**: `14.36`
  - The model processed around 14.36 samples per second during the evaluation phase.

- **Evaluation Steps Per Second**: `14.36`
  - The model completed about 14.36 evaluation steps per second, where each step typically involves processing a batch of samples.

- **Epochs**: `3`
  - The evaluation was done after the model had been trained for 3 full epochs.
